# Dataset Estadios
---------------
El objetivo de este código es filtrar y complementar la fuente de datos del proyecto, efectuando las siguientes transformaciones:
* Filtrado de la columna Confederation para conservar unicamente los estadios pertenecientes a la UEFA.
* Obtencion de las columnas de Longitud y Latitud por medio del uso de la API de Google Maps.

## Importar Librerias


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as py
import requests
import json
from config import gkey

## Creacion del dataframe a partir del archivo CSV

In [3]:
#Leer CSV y crear dataframe
stadiums_csv = "FootballStadiumsData.csv"
stadiums_df = pd.read_csv(stadiums_csv)
stadiums_df.dtypes

Confederation          object
Stadium                object
City                   object
HomeTeams              object
Capacity                int64
Country                object
IOC                    object
Population              int64
UCLChampions             bool
UELChampion              bool
UCLFinalStadium          bool
UELFinalStadium          bool
Capacidad/1000Habs    float64
dtype: object

In [10]:
stadiums_df.head()

,Confederation,Stadium,City,HomeTeams,Capacity,Country,IOC,Population,UCLChampions,UELChampion,UCLFinalStadium,UELFinalStadium,Capacidad/1000Habs
0,UEFA,Stadiumi Besëlidhja,Lezhë,Besëlidhja,7000,Albania,ALB,2876591,False,False,False,False,2.433436
1,UEFA,Stadiumi Flamurtari,Vlorë,Flamurtari Vlorë,8200,Albania,ALB,2876591,False,False,False,False,2.850596
2,UEFA,Stadiumi Laçi,Laçi,KF Laçi,5000,Albania,ALB,2876591,False,False,False,False,1.738169
3,UEFA,Stadiumi Niko Dovana,Durrës,Teuta,12040,Albania,ALB,2876591,False,False,False,False,4.185510
4,UEFA,Stadiumi Selman Stërmasi,Tirana,"KF Tirana, Dinamo, Partizani",9500,Albania,ALB,2876591,False,False,False,False,3.302520


## Filtrado de columna Confederation 

In [11]:
uefa_stadiums_df = stadiums_df.loc[stadiums_df['Confederation'] == 'UEFA']
uefa_stadiums_df.tail()

,Confederation,Stadium,City,HomeTeams,Capacity,Country,IOC,Population,UCLChampions,UELChampion,UCLFinalStadium,UELFinalStadium,Capacidad/1000Habs
2019,UEFA,Liberty Stadium,Swansea,Swansea City,21088,Wales,WAL,3125000,False,False,False,False,6.74816
2020,UEFA,Parc y Scarlets,Llanelli,"Scarlets, Llanelli RFC",14870,Wales,WAL,3125000,False,False,False,False,4.75840
2021,UEFA,Park Avenue,Aberystwyth,Aberystwyth Town,5000,Wales,WAL,3125000,False,False,False,False,1.60000
2022,UEFA,Millennium Stadium,Cardiff,Sin Equipo,74500,Wales,WAL,3125000,False,False,True,False,23.84000
2023,UEFA,Racecourse Ground,Wrexham,Wrexham AFC,15850,Wales,WAL,3125000,False,False,False,False,5.07200


In [12]:
uefa_stadiums_df.reset_index(inplace=True, drop=True)
uefa_stadiums_df.tail()

,Confederation,Stadium,City,HomeTeams,Capacity,Country,IOC,Population,UCLChampions,UELChampion,UCLFinalStadium,UELFinalStadium,Capacidad/1000Habs
1359,UEFA,Liberty Stadium,Swansea,Swansea City,21088,Wales,WAL,3125000,False,False,False,False,6.74816
1360,UEFA,Parc y Scarlets,Llanelli,"Scarlets, Llanelli RFC",14870,Wales,WAL,3125000,False,False,False,False,4.75840
1361,UEFA,Park Avenue,Aberystwyth,Aberystwyth Town,5000,Wales,WAL,3125000,False,False,False,False,1.60000
1362,UEFA,Millennium Stadium,Cardiff,Sin Equipo,74500,Wales,WAL,3125000,False,False,True,False,23.84000
1363,UEFA,Racecourse Ground,Wrexham,Wrexham AFC,15850,Wales,WAL,3125000,False,False,False,False,5.07200


In [13]:
#Salvar DataFrame en archivo CSV (Checkpoint)
uefa_stadiums_df.to_csv('UEFA_Stadiums_Data.csv')

## Recoleccion de Latitud y Longitud de los estadios UEFA
Anteriormente Google APIs tenia un limite de procesamiento de 1000 requests por dia, por lo que este proceso se efectuo en dos fases: primero se procesaron las primeras 1000 filas, y despues las siguientes 363. Sin embargo, el limite ha sido removido por parte de Google por lo que el procesamiento pudo haberse efectuado en una sola fase, pero al no estar al tanto de esta situacion, lo he realizado en dos fases.

In [22]:
#Variables a utilizar durante la recoleccion de datos
processed = 0
notfound = 0

#Bucle para recolectar Lat/Long de los estadios UEFA
for (idx, row) in stadiums_uefa_geo_df.iterrows():
    try:
        #Guarda el valor de la fila actual para cada columna
        stadium = (row.loc['Stadium'])
        city = (row.loc['City'])
        #URL de Google APIs de donde seran recolectadas las Latitudes/Longitudes
        query_url = "https://maps.googleapis.com/maps/api/geocode/json?address="+str(stadium)+","+str(city)+"&key="+str(gkey)
        #JSON request
        response = requests.get(query_url).json()
        #Variables donde se guardaran los valores 
        lat = response["results"][0]["geometry"]["location"]["lat"]
        lng = response["results"][0]["geometry"]["location"]["lng"]
        #Se crean las columnas y se guardan los respectivos valores en la fila actual
        stadiums_uefa_geo_df.at[idx, 'Latitude'] = lat
        stadiums_uefa_geo_df.at[idx, 'Longitude'] = lng
        #Se imprime la informacion para verificar que los valores se procesaron correctamente
        print(str(stadium)+","+str(city)+":"+str(lat)+","+str(lng))
        #Conteo para validar cuantos estadios se han procesado
        processed += 1
    #Manejo de errores en caso de que algun estadio no sea encontrado
    except:
        #Imprimir cuando una ciudad no sea encontrada
        print("City not found")
        #Conteo para validar cuantas ciudades no fueron encontradas
        notfound += 1
        continue
#Impresion para validar el conteo final de ciudades procesadas y no encontradas
print(str(processed)+" stadiums were processed. "+str(notfound)+" cities were not found.")

Estádio Municipal de Braga,Braga:41.5619546,-8.430493799999999
Estádio Municipal de Lagos,Lagos:37.1166053,-8.6775614
Estádio Municipal de Portimão,Portimão:37.1357173,-8.540002099999999
Estádio Nacional do Jamor,Lisbon:38.7086142,-9.2608892
Estádio Pina Manique,Lisbon:38.7373352,-9.2038256
Estádio Varzim SC,Póvoa de Varzim:41.3877581,-8.7725409
Cluj Arena,Cluj-Napoca:46.7684775,23.572416
Stadionul Areni,Suceava:47.6393091,26.2466554
Stadionul Avântul,Reghin:46.7809041,24.7106513
Stadionul Ceahlăul,Piatra Neamţ:46.9385675,26.3543244
Stadionul Central,Ovidiu:44.23636,28.5656279
Stadionul Concordia,Chiajna:44.4628498,25.9755806
Stadionul Cotroceni,Bucarest:44.4276835,26.0749598
Stadionul Dan Paltinisanu,Timişoara:45.7404604,21.2440338
Stadionul Dinamo,Bucarest:44.4550418,26.1023613
Stadionul Dr. Constantin Rădulescu,Cluj-Napoca:46.7796821,23.5772724
Stadionul Emil Alexandrescu,Iași:47.1845007,27.5612095
Stadionul Farul,Constanţa:44.1972911,28.6391551
Stadionul Flacăra,Moreni:44.9813823,2

Ljudski Vrt,Maribor:46.5629569,15.6406021
Mestni Stadion Lendava,Lendava:46.5593511,16.4507065
Športni Park Domžale,Domžale:46.1368957,14.6018042
Športni Park Nova Gorica,Nova Gorica:45.9558916,13.6407238
Stadion Bežigrad,Ljubljana:46.0691849,14.508591
Stadion Fazanerija,Murska Sobota:46.6686854,16.1577732
Stadion Matija Gubec,Krško:45.9488692,15.4875783
Stadion Primorje,Ajdovščina:45.887827,13.8938999
Stadion Stožice,Ljubljana:46.0804912,14.5242148
Stadion Z’dežele,Celje:46.2463529,15.2707258
Stadion ŽŠD,Ljubljana:46.06977699999999,14.499189
Abanca Balaídos,Vigo:42.212398,-8.740087299999999
Abanca-Riazor,La Coruńa:43.3687184,-8.4174835
Camp d'Esports,Lleida:41.6212978,0.6142732
Camp Municipal Narcís Sala,Barcelona:41.4288885,2.1930877
Camp Nou,Barcelona:41.380896,2.1228198
Campo de Fútbol de Vallecas,Madrid:40.3918127,-3.6586403
CIAT,Santa Cruz de Tenerife:28.4458593,-16.3077969
Coliseum Alfonso Pérez,Getafe:40.3257267,-3.7149399
Complejo Deportivo la Fuensanta,Cuenca:40.0764411,-2.14

Sivas Arena,Sivas:39.72862980000001,36.9840482
Timsah Arena,Bursa:40.21081,29.0094247
Torku Arena,Konya:37.9462451,32.48801
Türk Telekom Stadium,Istanbul:41.1180768,29.04825409999999
Sukru Saracoglu Stadyumu,Istanbul:40.9877414,29.0368233
Vodafone Park,Istanbul:41.0393954,28.9941087
Yeni Eskişehir Stadyumu,Eskişehir:39.7623815,30.4679136
Yeni Malatya Stadyumu,Malatya:38.3364112,38.4464615
Yeni Sakarya Stadyumu,Sakarya:40.7804295,30.4326372
Yusuf Ziya Öniş Stadyumu,Istanbul:41.1727731,29.0502165
Zeytinburnu Stadyumu,Istanbul:40.994299,28.9113348
Arena Lviv,Lviv:49.7748575,24.0277051
Dnipro Arena,Dnipropetrovsk:48.4603848,35.0324394
Donbass Arena,Donieck:48.0211066,37.8102215
Lafort Arena,Dobromyl:49.5761119,22.8016913
NSC Olimpiyskiy,Kiev:50.4334058,30.521856
Obolon Arena,Kiev:50.5270975,30.5075239
RSK Olimpiyskyi,Donieck:48.0192876,37.8030247
Sławutycz Arena,Zaporizhia:47.859427,35.097028
Sportiwnyj Kompleks Podilya,Khmelnytskyi:49.422983,26.9871331
Stadion Awangard,Luhansk:48.574041,3

In [23]:
stadiums_uefa_geo_df.tail(20)

,Confederation,Stadium,City,HomeTeams,Capacity,Country,IOC,Population,UCLChampions,UELChampion,UCLFinalStadium,UELFinalStadium,Capacidad/1000Habs,Latitude,Longitude
1344,UEFA,Stadion Illichivets,Mariupil,Illichivets,12680,Ukraine,UKR,42418235,False,False,False,False,0.298928,47.143902,37.558466
1345,UEFA,Stadion im W. Łobanowskiego,Kiev,Dynamo,16873,Ukraine,UKR,42418235,False,False,False,False,0.397777,50.450401,30.535101
1346,UEFA,Stadion Jubilejnyj,Sumy,FK Sumy,26000,Ukraine,UKR,42418235,False,False,False,False,0.612944,50.902773,34.799880
1347,UEFA,Stadion Lokomotyv,Simferopol,Tavria,19978,Ukraine,UKR,42418235,False,False,False,False,0.470977,44.943657,34.089465
1348,UEFA,Stadion Metalist,Kharkiv,Shakhtar,38613,Ukraine,UKR,42418235,False,False,False,False,0.910292,49.980878,36.261548
1349,UEFA,Stadion Metalurga,Krivyi Rih,Krywbas,29783,Ukraine,UKR,42418235,False,False,False,False,0.702127,47.910483,33.391783
1350,UEFA,Stadion Meteor,Dnipropetrovsk,Dnipro,24381,Ukraine,UKR,42418235,False,False,False,False,0.574776,48.436429,35.002930
1351,UEFA,Stadion NTK,Kiev,Sin Equipo,1678,Ukraine,UKR,42418235,False,False,False,False,0.039558,50.429063,30.525241
1352,UEFA,Stadion Ruch,Ivano-Frankivsk,Prykarpattya,15000,Ukraine,UKR,42418235,False,False,False,False,0.353622,48.908097,24.695923
1353,UEFA,Stadion Stali,Altchevs'k,Stal,12000,Ukraine,UKR,42418235,False,False,False,False,0.282897,48.474177,38.797558


In [24]:
#Salva el DataFrame en un archivo CSV
stadiums_uefa_geo_df.to_csv('UEFA_Stadiums_Geo_Data.csv')